In [1]:
#main_taberconley_controls.do
#sample.do
#!pip install linearmodels
#!pip install econtools
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))

#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1
#id_auth =  econ.group_id(df, cols='authority_code')
#print(id_auth)
#print(df['id_auth'].values.tolist())

work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


In [9]:
#keep the observations need
#df = untouched 
df1 = df
df1 = df1[((df1['turin_co_sample'] == 1)|(df1['turin_pr_sample'] == 1))&((df1['post_experience']>=5)|(df1['post_experience'].isnull()==True)) &((df1['pre_experience']>=5)|(df1['pre_experience'].isnull()==True)) &(df1['missing']==0)]
#df_pr = df1[(df1['turin_co_sample'] != 1)&(df1['turin_pr_sample'] == 1)&(df1['post_experience']>=5) &(df1['pre_experience']>=5) &(df1['missing']==0)]
#df_co_pr = df1[(df1['turin_co_sample'] == 1)&(df1['turin_pr_sample'] == 1)&(df1['post_experience']>=5) &(df1['pre_experience']>=5) &(df1['missing']==0)]
#df1 = pd.concat([df_co,df_pr,df_co_pr])
df1= df1[(df1['ctrl_pop_turin_co_sample']==1) | (df1['ctrl_pop_turin_pr_sample']==1) | (df1['ctrl_exp_turin_co_sample']==1) | (df1['ctrl_exp_turin_pr_sample']==1) | (df1['ctrl_pop_exp_turin_co_sample']==1) | (df1['ctrl_pop_exp_turin_pr_sample']==1)]
print(df1.shape)

(2016, 93)


In [10]:
#reconstruct trend_pa (another version)
id_auth_group =  econ.group_id(df1, cols='id_auth')
id_auth_group['group_id'] = id_auth_group['group_id'] +1
#print(id_auth_group)

df1['id_auth_remained'] = 0
for i in range(len(df1)):
    for j in range(len(id_auth_group)):
        if df1.iloc[i,df1.columns.get_loc('id_auth')] == id_auth_group.loc[j, 'id_auth']:
            df1.iloc[i,df1.columns.get_loc('id_auth_remained')] = id_auth_group.loc[j, 'group_id']
id_auth_remained_dum = pd.get_dummies(df1['id_auth_remained']).rename(columns=lambda x: 'id_auth_remained' + str(x))
df1 = pd.concat([df1, id_auth_remained_dum],axis = 1)

for i in range(len(id_auth_group)):
    df1['trend_pa_remained'+str(i+1)] =0
    col_num_tr = df1.columns.get_loc('trend_pa_remained'+str(i+1))
    col_num_id = df1.columns.get_loc('id_auth_remained'+str(i+1))
    for j in range(len(df1)):
        if df1.iloc[j, col_num_id]==1 and df1.iloc[j, df1.columns.get_loc('authority_code')]!= 3090272 and df1.iloc[j,df1.columns.get_loc('authority_code')]!=3070001:
            df1.iloc[j ,col_num_tr] = df1.iloc[j, df1.columns.get_loc('trend')]
    df1.drop(['id_auth_remained'+str(i+1)], axis = 1)

In [32]:
print(df1[df1['work_category']==""].index)

Int64Index([], dtype='int64')


In [79]:
#create dummies pa-year
from statsmodels.stats.outliers_influence import variance_inflation_factor
# mutlicollinearity function
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)
treatment = ['turin_co_sample'] #, 'turin_pr_sample']
group = [ 'ctrl_exp'] #, 'ctrl_pop', 'ctrl_pop_exp']
outcomes = ['discount'] #, 'delay_ratio', 'overrun_ratio', 'days_to_award'] 
c_outcomes = 1
i = 5

for t in treatment:
    for g in group:
        for o in outcomes:
            #matrix = nc_ + o + t 
            df1 = df1[(df1[t]==1) & (df1[g +'_' + t]==1) & (df1['post_experience']>=i) & (df1['pre_experience']>=i)& (df1['post_experience'].isnull()==False) & (df1['pre_experience'].isnull()==False) & (df1['missing']==0) & (df1[o].isnull()==False) & (df1['fiscal_efficiency'].isnull()==False) & (df1['reserve_price'].isnull()==False)&(df1['municipality'].isnull()==False)]
            df1.sort_values(by = 'authority_code', ascending = True)
            #for j in range(len(df_tab5)):
            #    if df_tab5.loc[j,'authority_code']!=df[[:,j],'authority_code']:
            #        df_tab5['ind'] = 1
            #line61-64 skip
            
            #create dummies for administration-year pairs 
            all_years = df1['year'].unique()
            all_authorities = df1['authority_code'].unique()
            auth_year_reg_col = []
            for auth in all_authorities:
                for yr in all_years:
                    df1['auth_year_' + str(int(auth))+'_' + str(int(yr))] = 0
                    auth_year_reg_col.append('auth_year_' + str(int(auth))+'_' + str(int(yr)))
                    df1.loc[(df1['year']==yr) & (df1['authority_code']==auth), 'auth_year_' + str(int(auth))+'_' + str(int(yr)) ] = 1
           
            # create dummies for work category
            all_categories = df1['work_category'].unique()
            i = 1
            for cat in all_categories:
                df1['cat_'+cat] = 0
                df1.loc[df1['work_category']==cat, 'cat'+cat] =1
                if i == 1:
                #line91
                    i = i+1
                #else:
                #line 95
       
            ### Regression first stage 
            #make a exog list
            reg_col = []
            for i in work_list:
                reg_col.append(i)
            for j in year_list:
                reg_col.append(j)
            
            exog_var = ['fpsb_auction','reserve_price','municipality']
            exog = exog_var + reg_col 
            #i_authority_code 안 만드는 이유 = fe_name에 들어가있어서

            #vif cal
            #check multicollinearity
            X = df1.loc[:,exog]
            vif = calc_vif(X)
            #print(vif)


            #delete from col list
            for i in range(len(vif)):
                if np.isnan(vif.loc[i, 'VIF']) == True:
                    reg_col.remove(vif.loc[i, 'variables'])
                elif vif.loc[i,'VIF'] > 10:
                    for j in exog_var:
                        if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                            exog_var.remove(vif.loc[i,'variables'])
                
            exog = exog_var + reg_col
            exog.remove(2000)
            exog.remove('OG01')
            exog.remove('municipality')
    
            #1. reg
            fe_reg_1 = mt.reg(df1, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
            
            #2. reg
            fe_reg_2 = mt.reg(df1, o, exog, fe_name = 'authority_code', cluster = 'authority_code')
            
            #3. reg
            reg_col = auth_year_reg_col
            for cat in all_categories:
                reg_col.append('cat_'+cat)
            exog_var = ['reserve_price','municipality','fiscal_efficiency']
            exog = exog_var + reg_col
            
            X = df1.loc[:,exog]
            vif = calc_vif(X)
            #print(vif)
            for i in range(len(vif)):
                if np.isnan(vif.loc[i, 'VIF']) == True:
                    reg_col.remove(vif.loc[i, 'variables'])
                '''elif vif.loc[i,'VIF'] > 10:
                    for j in exog:
                        exog_var.remove(vif.loc[i,'variables'])'''
                
            exog = exog_var + reg_col
            exog.remove('municipality')
            
            fe_reg_3 = mt.reg(df1, o, exog, cluster = 'auth_anno')
            print(fe_reg_1, fe_reg_2, fe_reg_3)

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			1262
R-squared:		0.6393
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se      t   p>t  CI_low CI_high
fpsb_auction  11.992 1.315  9.117 0.000   9.382  14.602
reserve_price  0.000 0.000  5.993 0.000   0.000   0.000
OG02           0.205 0.629  0.326 0.745  -1.043   1.452
OG03          -0.583 0.545 -1.070 0.287  -1.665   0.498
OG04          -4.754 3.344 -1.422 0.158 -11.389   1.881
OG06           0.852 0.655  1.300 0.196  -0.448   2.151
OG07           4.509 5.121  0.881 0.381  -5.651  14.668
OG08           0.801 1.606  0.499 0.619  -2.384   3.986
OG10           8.419 0.515 16.337 0.000   7.396   9.441
OG11           5.680 1.058  5.367 0.000   3.580   7.780
OG12           6.017 1.940  3.101 0.003   2.168   9.866
OS01          -2.534 1.204 -2.104 0.038  -4.923  -0.145
OS02           9.521 1.405  6.777 0.000   6.734  12.308
OS05 

C:\Users\thxkn\anaconda3\lib\site-packages\econtools\metrics\core.py:203: RuntimeWarning: invalid value encountered in sqrt
  se = pd.Series(np.sqrt(np.diagonal(vce)), index=vce.columns)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


In [ ]:
# create dummies for work category
all_categories = df['work_cateogory'].unique()

from statsmodels.stats.outliers_influence import variance_inflation_factor

i = 1
for cat in all_categories:
    df['cat_'+cat] = 0
    df.loc[df['work_category']==cat, 'cat'+cat] =1
    if i == 1:
        #line91
        i = i+1
    else:
        #line 95
        
# Regression first stage 
    # mutlicollinearity function
    def calc_vif(X):

        # Calculating VIF
        vif = pd.DataFrame()
        vif["variables"] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

        return(vif)

    #iteration
#for o in outcome: create dum pa-year이랑 연결됨
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
        
        reg_col.append('auth_year_' + auth + yr)
    #i_authority_code 안 만드는 이유 = fe_name에 들어가있어서
    exog_var = ['fpsb_auction','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)
    exog.remove('OG01')
    #exog.remove('municipality')
    
    #1. reg
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    #print(fe_reg) 
    if 'c_outcomes' == 1:
        #outreg2
    else:
        #outreg2
        
    #2. reg cluster(authority_code) line110
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'authority_code')
    if 'c_outcomes' == 1:
        #outreg
    else: 
        #outreg
    
    
    
    ####3. reg line 118
    def calc_vif(X):

        # Calculating VIF
        vif = pd.DataFrame()
        vif["variables"] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

        return(vif)

    #iteration
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in all_categories:
        reg_col.append('cat_'+i)
    for auth in all_authorities:
        for yr in all_years:
             reg_col.append('auth_year_' + auth + yr)
    
    exog_var = ['fiscal efficiency','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('municipality')
    
    #1. reg
    fe_reg = mt.reg(df_name, o, exog, cluster = 'auth_anno')
    
#line 120-121

In [ ]:
#create weights (123)


In [ ]:
#* Keep only beta coefficients for state*year terms *(130)

In [ ]:
#* Core conley-taber method*

In [ ]:
#/* Predict residuals from regression */(137)

In [ ]:
#/* Create d tilde variable*/
df_tab5 = df_tab5.sort_values(by = 'year', ascending = True)
df_tab5.loc[(df['authority_code']==3090272) | (df['authority_code']==3070001),'djtga'] = df['authority_code'].mean()
df_tab5['djt'] = df['djtga'].sum()

df_tab5 = df_tab5.sort_values(by = 'authority_code',ascending = True)
df_tab5['meandjt'] = df['djt'].mean()
df_tab5['tilde'] = df['djt'] - df['meandjt']

In [ ]:
#line148 Obtain difference in differences coefficient*/
#line149 re-normalize weights*/

df.loc[(df['authority_code']==3090272) | (df['authority_code']==3070001), 'tot_weights'] = df['weights']
df['new_weights'] = df['weights']/df['tot_weights']
mod_wls = sm.WLS(eta, dtil, weights=df['new_weights']).fit()
df.drop(['new_weights','tot_weights'])

In [ ]:
#/* Simulations for each public administration

#line160
for auth in all_authorities:


#line 177,178

In [ ]:
# Confidence intervals

#line 231